# Processing recipes   
Credits   
https://www.oreilly.com/library/view/python-data-science/9781491912126/

In [5]:
import pandas as pd
import numpy as np

In [1]:
!curl -O https://s3.amazonaws.com/openrecipes/20170107-061401-recipeitems.json.gz
!gunzip 20170107-061401-recipeitems.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.3M  100 29.3M    0     0  14.2M      0  0:00:02  0:00:02 --:--:-- 14.2M


In [12]:
!ls -la
print(pd.__version__)

total 295120
drwxr-xr-x  6 dantohe  staff        192 May 15 11:02 .
drwxr-xr-x  8 dantohe  staff        256 May 14 17:17 ..
drwxr-xr-x  4 dantohe  staff        128 May 15 10:52 .ipynb_checkpoints
-rw-r--r--  1 dantohe  staff  141698284 May 15 10:51 20170107-061401-recipeitems.json
-rw-r--r--  1 dantohe  staff      22766 May 15 11:02 processing_recipes.ipynb
-rw-r--r--  1 dantohe  staff      13917 May 15 10:48 vectorized_strings.ipynb
1.2.3


In [11]:
# read the entire file into a Python array
from io import StringIO
# newdf = pd.read_json(StringIO(temp))

with open('20170107-061401-recipeitems.json', 'r') as f:
    # Extract each line
    data = (line.strip() for line in f)
    # Reformat so each line is the element of a list
    data_json = "[{0}]".format(','.join(data))
# read the result as a JSON
recipes = pd.read_json(StringIO(data_json))

In [15]:
print(recipes.shape)
recipes.info()

(173278, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173278 entries, 0 to 173277
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   _id                 173278 non-null  object
 1   name                173278 non-null  object
 2   ingredients         173278 non-null  object
 3   url                 173278 non-null  object
 4   image               158278 non-null  object
 5   ts                  173278 non-null  object
 6   cookTime            117936 non-null  object
 7   source              173278 non-null  object
 8   recipeYield         165628 non-null  object
 9   datePublished       78110 non-null   object
 10  prepTime            130186 non-null  object
 11  description         158068 non-null  object
 12  totalTime           1570 non-null    object
 13  creator             395 non-null     object
 14  recipeCategory      388 non-null     object
 15  dateModified        161 non-null     o

In [16]:
recipes.iloc[0]


_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

In [17]:
recipes.ingredients.str.len().describe()


count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

In [18]:
recipes.name[np.argmax(recipes.ingredients.str.len())]


'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

In [19]:
recipes.description.str.contains('[Bb]reakfast').sum()


3524

In [20]:
recipes.ingredients.str.contains('[Cc]innamon').sum()


10526

In [21]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

In [25]:
import re
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE)) for spice in spice_list))
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [23]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

In [24]:
recipes.name[selection.index]


2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object